In [1]:
import pandas as pd
import mne_bids
import matplotlib.pyplot as plt
import mne
import os
import numpy as np

/Users/anubhavsarkar/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
subs = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27']
sess = ['0','1']
tasks = ['0','1','2','3']

In [ ]:
ica = mne.preprocessing.ICA(n_components=20, random_state=97, max_iter=800)
results_folder = 'results'
os.makedirs(results_folder, exist_ok=True)
for sub in subs:
    for ses in sess:
        for task in tasks:
            bids_path = mne_bids.BIDSPath(
            subject=sub,
            session=ses,
            task=task,
            datatype="meg",
            root='',
            )
            
            try:
                raw = mne_bids.read_raw_bids(bids_path)
            except FileNotFoundError:
                print("missing", sub, ses, task)
                continue
            
            raw.load_data().filter(0.5, 30.0, n_jobs=1)

            raw = raw.pick_types(meg=True, misc=False, eeg=False, eog=False, ecg=False)
            ica.fit(raw)
            raw.load_data()

            ica.apply(raw)
            plot_name = f'sub-{sub}_ses-{ses}_task-{task}.png'
            plot_path = os.path.join(results_folder, plot_name)

            # Save the raw.plot() to the "results" folder with the proper name
            raw.plot(show=False)  # Set show=False to prevent displaying the plot
            plt.savefig(plot_path)
            plt.close()  # Close the plot after saving to prevent multiple plots in memory

            print(f"Plot saved for sub-{sub}, ses-{ses}, task-{task} in {plot_path}")




In [4]:
import os
import shutil
image_folder = "results"
label_mapping = {}
for filename in os.listdir(image_folder):
    if filename.endswith(".png"):  # Make sure to filter only image files
        image_name = os.path.splitext(filename)[0]  # Extract image name without extension
        label = image_name[-1]  # Assuming the label is the last character before the extension
        
        # Create a mapping between image number and label
        label_mapping[f"results/{filename}"] = f"{label}"
# Now you have a dictionary where keys are image numbers and values are corresponding labels


In [5]:

image_folder = "results"
output_folder = "organized_results"  # Specify the name of the output folder

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Mapping between label and class subfolder
label_to_class = {
    "0": "lw1",
    "1": "cable_spool_fort",
    "2": "easy_money",
    "3": "The_Black_Widow"
    # Add more labels and corresponding class subfolders if needed
}

# Iterate through the label_mapping dictionary and organize images
for image_path, label in label_mapping.items():
    class_folder = os.path.join(output_folder, label_to_class[label])
    os.makedirs(class_folder, exist_ok=True)

    # Move the image to the corresponding class subfolder
    shutil.move(image_path, os.path.join(class_folder, os.path.basename(image_path)))

# Now your images are organized into class subfolders within the 'organized_results' folder